## 1. Create the main gempyor inference object from the config with all subpops
(you might need to comment the subpop_placeholder thing)

Rsync the results form the cluster
```bash
rsync -avz --prune-empty-dirs --include='*/' --include="*.h5" --include="*.pdf*" --include="*.yml" --exclude='*' chadi@longleaf.unc.edu:/work/users/c/h/chadi/flepiMoP_Notebooks/Flu_USA .
```

In [ ]:
import gempyor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
import copy

# import seaborn as sns
import matplotlib._color_data as mcd
import pyarrow.parquet as pq
import click
import subprocess
import dask.dataframe as dd
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
from matplotlib.backends.backend_pdf import PdfPages
from gempyor.utils import config, as_list
import os
import multiprocessing as mp
import pandas as pd
import pyarrow.parquet as pq
import xarray as xr
from gempyor import config, model_info, outcomes, seir, inference_parameter, logloss, inference
from gempyor.inference import GempyorInference
import gempyor.postprocess_inference
import multiprocessing, shutil

import os
from multiprocessing import cpu_count

# disable  operations using the MKL linear algebra.
os.environ["OMP_NUM_THREADS"] = "1"
import emcee

from multiprocessing import Pool


# Flu SMH
project_path = "Flu_USA/"
config_filepath = f"config_SMH_Flu_2024_R1_allflu_medVax_H3_training_multiseason_emcee_difflocvarseas.yml"


run_id = config_filepath.split("/")[-1].split(".")[0].replace("config_", "")
nwalkers = 10
niter = 4
nsamples = 50 # number of likelyhood eval to write to disk...
nthin=5


#### START COPY PAST FROM flepiMoP/flepimop/gempyor_pkg/src/gempyor/calibrate.py
ncpu = cpu_count()
print(f"found {ncpu} CPUs, using these")

gempyor_inference = GempyorInference(
        config_filepath=config_filepath,
        run_id=run_id,
        prefix=None,
        first_sim_index=1,
        rng_seed=None,
        nslots=1,
        inference_filename_prefix="global/final/",  # usually for {global or chimeric}/{intermediate or final}
        inference_filepath_suffix="",  # usually for the slot_id
        out_run_id=None,  # if out_run_id is different from in_run_id, fill this
        out_prefix=None,  # if out_prefix is different from in_prefix, fill this
        path_prefix=project_path,  # in case the data folder is on another directory
        autowrite_seir=False,
)
#inferpar = inference_parameter.InferenceParameters(global_config=config, modinf=modinf)
p0 = gempyor_inference.inferpar.draw_initial(n_draw=nwalkers)
for i in range(nwalkers):
    assert  gempyor_inference.inferpar.check_in_bound(proposal=p0[i]), "The initial parameter draw is not within the bounds, check the perturbation distributions"

nsubpop = len(gempyor_inference.modinf.subpop_struct.subpop_names)
subpop_names = gempyor_inference.modinf.subpop_struct.subpop_names

print(gempyor_inference.logloss)
print(gempyor_inference.inferpar)

found 12 CPUs, using these
  gempyor >> Running ***DETERMINISTIC*** simulation;
  gempyor >> ModelInfo USA_none_med; index: 1; run_id: SMH_Flu_2024_R1_allflu_medVax_H3_training_multiseason_emcee_difflocvarseas,
  gempyor >> prefix: USA_none_med/SMH_Flu_2024_R1_allflu_medVax_H3_training_multiseason_emcee_difflocvarseas/;
Loaded subpops in loaded relative probablity file: 153 Intersect with seir simulation:  153 kept
Running Gempyor Inference
LogLoss: 1 statistics and 37638 data points,number of NA for each statistic: 
source           0
season           0
incidH           0
incidH_AllFlu    0
dtype: int64
InferenceParameters: with 1377 parameters: 
    seir_modifiers: 1377 parameters

LogLoss: 1 statistics and 37638 data points,number of NA for each statistic: 
source           0
season           0
incidH           0
incidH_AllFlu    0
dtype: int64
InferenceParameters: with 1377 parameters: 
    seir_modifiers: 1377 parameters



## Load each state-specific config and h5 files and run nsamples iteration for each, to get model output

In [ ]:
shutil.rmtree("model_output/", ignore_errors=True)
shutil.rmtree(os.path.join(project_path, "model_output/"), ignore_errors=True)
shutil.rmtree(os.path.join(project_path, "/model_output/"), ignore_errors=True)
states = [sp.split("_")[0] for sp in subpop_names]
states = list(set(states))
nsamples=10
chain_index = -1 # -1 for last
for i, sp in enumerate(states):
    print(f"Subpop: {i} {sp}", end=" ")
    try:
        filename = f"Flu_USA/SMH_Flu_2024_R1_allflu_medVax_H3_training_multiseason_emcee_difflocvarseas_{sp}-*.h5"
        # find a file that matches the pattern
        import glob
        filename = glob.glob(filename)
        if len(filename) == 0:
            print(f"File {filename} does not exist")
            continue
        filename = filename[0]

        # check if the file exists
        if not os.path.exists(filename):
            print(f"File {filename} does not exist")
            continue
        else: 
            print("lets go")
        sampler = emcee.backends.HDFBackend(filename, read_only=True)

        max_indices = np.argsort(sampler.get_log_prob()[chain_index, :])[-nsamples:]
        print(sampler.get_chain().shape)
        chains = sampler.get_chain()[:chain_index, max_indices, :]        # the last iteration, for selected slots
        samples = sampler.get_chain()[chain_index, max_indices, :]  # the last iteration, for selected slots
        llik = sampler.get_log_prob()[:chain_index, max_indices]
        gempyor_inference.set_save(True)
        config_filepath = f"Flu_USA/config_SMH_Flu_2024_R1_allflu_medVax_H3_training_multiseason_emcee_difflocvarseas_{sp}.yml"
        run_id = "flu_" + sp
        gempyor_inference = GempyorInference(
            config_filepath=config_filepath,
            run_id=run_id,
            prefix=None,
            first_sim_index=1,
            rng_seed=None,
            nslots=1,
            inference_filename_prefix="global/final/",  # usually for {global or chimeric}/{intermediate or final}
            inference_filepath_suffix="",  # usually for the slot_id
            out_run_id=None,  # if out_run_id is different from in_run_id, fill this
            out_prefix=None,  # if out_prefix is different from in_prefix, fill this
            path_prefix=project_path,  # in case the data folder is on another directory
            autowrite_seir=False,
        )
        gempyor_inference.set_save(True)
        with multiprocessing.Pool(ncpu) as pool:
            results = pool.starmap(
                gempyor_inference.get_logloss_as_single_number, [(samples[i, :],) for i in range(len(max_indices))]
            )
    except Exception as e:
        print(f"Error: {e}")
        continue
    

In [4]:
results = {}
for fn in gempyor.utils.list_filenames(folder=project_path+"model_output/", filters=["hosp.parquet"]): ## TODO: here project path is correct, but use to concatenate path. the right way
    sp = fn.split("_")[-1].split(".")[0]
    if sp in results:
        results[sp].append(gempyor.read_df(fn))
    else:
        results[sp] = [gempyor.read_df(fn)]
    df = gempyor.read_df(fn)
    df = df.set_index("date", drop=True)
print(len(results))

51


In [5]:
list_of_dfs = []
for i in range(nsamples):
    to_concat = []
    for sp in results:
        to_concat.append(results[sp][i])
    df = pd.concat(to_concat, axis=0)
    list_of_dfs.append(df.set_index("date", drop=True))
results=list_of_dfs

## reload the main gempyor object from the config that has all subpop to plot the all together

In [ ]:
config_filepath = f"config_SMH_Flu_2024_R1_allflu_medVax_H3_training_multiseason_emcee_difflocvarseas.yml"

# Disparities
#project_path = "COVID19_Disparities"
#config_filepath = f"config_SMH_Rdisparity_phase_one_phase1_blk1_limitIFR_CA-NC_emcee.yml"

run_id = config_filepath.split("/")[-1].split(".")[0].replace("config_", "")



#### START COPY PAST FROM flepiMoP/flepimop/gempyor_pkg/src/gempyor/calibrate.py
ncpu = cpu_count()
print(f"found {ncpu} CPUs, using these")

gempyor_inference = GempyorInference(
        config_filepath=config_filepath,
        run_id=run_id,
        prefix=None,
        first_sim_index=1,
        rng_seed=None,
        nslots=1,
        inference_filename_prefix="global/final/",  # usually for {global or chimeric}/{intermediate or final}
        inference_filepath_suffix="",  # usually for the slot_id
        out_run_id=None,  # if out_run_id is different from in_run_id, fill this
        out_prefix=None,  # if out_prefix is different from in_prefix, fill this
        path_prefix=project_path,  # in case the data folder is on another directory
        autowrite_seir=False,
)

found 12 CPUs, using these
  gempyor >> Running ***DETERMINISTIC*** simulation;
  gempyor >> ModelInfo USA_none_med; index: 1; run_id: SMH_Flu_2024_R1_allflu_medVax_H3_training_multiseason_emcee_difflocvarseas,
  gempyor >> prefix: USA_none_med/SMH_Flu_2024_R1_allflu_medVax_H3_training_multiseason_emcee_difflocvarseas/;
Loaded subpops in loaded relative probablity file: 153 Intersect with seir simulation:  153 kept
Running Gempyor Inference
LogLoss: 1 statistics and 37638 data points,number of NA for each statistic: 
source           0
season           0
incidH           0
incidH_AllFlu    0
dtype: int64
InferenceParameters: with 1377 parameters: 
    seir_modifiers: 1377 parameters



In [ ]:

gempyor.postprocess_inference.plot_fit(modinf=gempyor_inference.modinf, loss=gempyor_inference.logloss, list_of_df=results, save_to=f"{run_id}_fit_gtdates.pdf")
gempyor.postprocess_inference.plot_fit(modinf=gempyor_inference.modinf, loss=gempyor_inference.logloss,plot_projections=True, list_of_df=results, save_to=f"{run_id}_fit_proj.pdf")